<a href="https://colab.research.google.com/github/BrahiamAcosta/Competencia-Kaggle-Modelos1/blob/main/03%20-%20modelo%20eliminando%20variables%20y%20con%20seleccion%20automatica%20de%20la%20plantilla.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os

os.environ["KAGGLE_CONFIG_DIR"] = "."


!kaggle competitions download -c udea-ai4eng-20242
!unzip udea-ai4eng-20242.zip
!rm udea-ai4eng-20242.zip

 89% 18.0M/20.1M [00:01<00:00, 13.1MB/s]
100% 20.1M/20.1M [00:01<00:00, 10.7MB/s]
Archive:  udea-ai4eng-20242.zip
  inflating: submission_example.csv  
  inflating: test.csv                
  inflating: train.csv               


#Preprocesado:
- Variables binarias a 0 y 1
- Variables categoricas a one hot encoding
- Variables que se eliminan:id,periodo,educación de padre y madre y el departamento de estudio

In [ ]:
!pip install category_encoders
import pandas as pd
from category_encoders import TargetEncoder
from sklearn.model_selection import train_test_split
import numpy as np

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 2.0 MB/s eta 0:00:00


In [ ]:
def clean_dataset(d):
  def to_onehot(x):
    values = np.unique(x)
    r = np.r_[[np.argwhere(i==values)[0][0] for i in x]]
    return np.eye(len(values))[r].astype(int)

  def replace_columns_with_onehot(d, col):
      k = to_onehot(d[col].values)
      r = pd.DataFrame(k, columns=["%s_%d"%(col, i) for i in range(k.shape[1])], index=d.index).join(d)
      del(r[col])
      return r

  #Variables a eliminar
  del(d['PERIODO'])
  del(d['FAMI_EDUCACIONMADRE'])
  del(d['FAMI_EDUCACIONPADRE'])
  del(d['ESTU_PRGM_DEPARTAMENTO'])

  d.FAMI_TIENEINTERNET = d.FAMI_TIENEINTERNET.map({"No": 0, "Si": 1})
  d.ESTU_PAGOMATRICULAPROPIO = d.ESTU_PAGOMATRICULAPROPIO.map({"No": 0, "Si": 1})

  #Obtener las probabilidades de ocurrencia para las varibales binarias
  conteo = d['ESTU_PAGOMATRICULAPROPIO'].value_counts()
  pago_mtr_prob = [(conteo / len(d))[0], (conteo / len(d))[1]]

  conteo = d['FAMI_TIENEINTERNET'].value_counts()
  internet_prob = [(conteo / len(d))[0], (conteo / len(d))[1]]

  #Fillna
  d.ESTU_VALORMATRICULAUNIVERSIDAD = d.ESTU_VALORMATRICULAUNIVERSIDAD.fillna(d.ESTU_VALORMATRICULAUNIVERSIDAD.mode()[0])
  d.ESTU_HORASSEMANATRABAJA = d.ESTU_HORASSEMANATRABAJA.fillna(d.ESTU_HORASSEMANATRABAJA.mode()[0])
  d.FAMI_ESTRATOVIVIENDA = d.FAMI_ESTRATOVIVIENDA.fillna(d.FAMI_ESTRATOVIVIENDA.mode()[0])
  d.FAMI_TIENEINTERNET = d.FAMI_TIENEINTERNET.fillna(d.FAMI_TIENEINTERNET.mode()[0])
  d.ESTU_PAGOMATRICULAPROPIO = d.ESTU_PAGOMATRICULAPROPIO.fillna(d.ESTU_PAGOMATRICULAPROPIO.mode()[0])
  d.programa_encoded = d.programa_encoded.fillna(d.programa_encoded.mean())


  #Onehot
  d = replace_columns_with_onehot(d, "ESTU_VALORMATRICULAUNIVERSIDAD")
  d = replace_columns_with_onehot(d, "ESTU_HORASSEMANATRABAJA")
  d = replace_columns_with_onehot(d, "FAMI_ESTRATOVIVIENDA")

  return d


##Cargar los datos de train y calibrar el modelo

In [ ]:
dtr = pd.read_csv('train.csv')
dtr.head()

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO,RENDIMIENTO_GLOBAL
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Postgrado,No,medio-alto
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Técnica o tecnológica incompleta,No,bajo
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,No,bajo
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Secundaria (Bachillerato) completa,No,alto
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Primaria completa,No,medio-bajo


In [ ]:
#mapear rendimiento en el train
rendimiento_dict = {"bajo":1, "medio-bajo":2, "medio-alto":3, "alto":4}
dtr.RENDIMIENTO_GLOBAL = dtr.RENDIMIENTO_GLOBAL.map(rendimiento_dict)

In [ ]:
#Calibrar el TargetEncoder con los datos de entrenamiento
import pandas as pd
from category_encoders import TargetEncoder
from sklearn.model_selection import train_test_split

X_train_TE = dtr[['ESTU_PRGM_ACADEMICO']]
y_train_TE = dtr['RENDIMIENTO_GLOBAL']

# Inicializar TargetEncoder
encoder = TargetEncoder(cols=['ESTU_PRGM_ACADEMICO'], smoothing=0.5)
encoder.fit(X_train_TE, y_train_TE)

X_train_encoded = encoder.transform(X_train_TE)

dtr['programa_encoded'] = X_train_encoded['ESTU_PRGM_ACADEMICO']

# Agregar la columna de encoding en el DataFrame original y eliminar la columna original
dtr = dtr.drop(columns=['ESTU_PRGM_ACADEMICO'])

In [ ]:
dtr = clean_dataset(dtr)

In [ ]:
#Extraer X y y para calibrar el modelo
Xtr = dtr.drop(columns=['RENDIMIENTO_GLOBAL'])
ytr = dtr.RENDIMIENTO_GLOBAL

In [ ]:
#Eliminar el Id de lso datos de prueba para entrenar el modelo
del(Xtr['ID'])

##Aplicar PCA para reducir la dimensionalidad de los datos y el ruido

In [ ]:
Xtr.shape

(692500, 23)

In [ ]:
from sklearn.decomposition import PCA

# Reducir a un número menor de dimensiones
pca = PCA(n_components=10)
X_reduced = pca.fit_transform(Xtr)

##Escoger el mejor estimador y ajustarlo:

In [ ]:
!pip install tqdm

In [ ]:
from sklearn.model_selection import cross_validate, ShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from tqdm import tqdm

def report_cv_score(z):
    print ("test score   %.3f (±%.4f) with %d splits"%(np.mean(z["test_score"]), np.std(z["test_score"]), len(z["test_score"])))
    print ("train score  %.3f (±%.4f) with %d splits"%(np.mean(z["train_score"]), np.std(z["train_score"]), len(z["train_score"])))


estimator1 = LogisticRegression()

estimator2 = DecisionTreeClassifier()

estimator3 = RandomForestClassifier()


zscores = []
estimators = [estimator1, estimator2, estimator3]
for estimator in tqdm(estimators,desc='Evaluando estimadores'):
    print("--")
    z = cross_validate(estimator, X_reduced, ytr, return_train_score=True, return_estimator=False,
                       scoring='accuracy', cv=ShuffleSplit(n_splits=3, test_size=0.3))
    report_cv_score(z)
    zscores.append(np.mean(z["test_score"]))
best = np.argmax(zscores)
print ("selecting ", best)
best_estimator = estimators[best]
print ("\nselected model")
print (best_estimator)

Evaluando estimadores:   0%|          | 0/3 [00:00<?, ?it/s]

--


Evaluando estimadores:  33%|███▎      | 1/3 [00:08<00:16,  8.48s/it]

test score   0.402 (±0.0007) with 3 splits
train score  0.402 (±0.0002) with 3 splits
--


Evaluando estimadores:  67%|██████▋   | 2/3 [00:37<00:20, 20.31s/it]

test score   0.391 (±0.0010) with 3 splits
train score  0.510 (±0.0003) with 3 splits
--


Evaluando estimadores: 100%|██████████| 3/3 [10:31<00:00, 210.39s/it]

test score   0.394 (±0.0000) with 3 splits
train score  0.510 (±0.0002) with 3 splits
selecting  0

selected model
LogisticRegression()


In [ ]:
best_estimator.fit(X_reduced, ytr)

LogisticRegression()

In [ ]:
dts = pd.read_csv('test.csv')
Xts = dts.drop(columns=['ID'])
del(Xts['Unnamed: 0'])
Xts.head()

,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO
0,20183,TRABAJO SOCIAL,BOLIVAR,Menos de 500 mil,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica completa,Primaria completa,Si
1,20203,ADMINISTRACION COMERCIAL Y DE MERCADEO,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Técnica o tecnológica completa,No
2,20212,INGENIERIA MECATRONICA,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,0,Estrato 3,Si,Secundaria (Bachillerato) incompleta,Secundaria (Bachillerato) completa,No
3,20195,CONTADURIA PUBLICA,SUCRE,Entre 1 millón y menos de 2.5 millones,Entre 21 y 30 horas,Estrato 1,No,Primaria incompleta,Primaria incompleta,No
4,20212,ADMINISTRACION DE EMPRESAS,ATLANTICO,Entre 2.5 millones y menos de 4 millones,Entre 11 y 20 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,No


In [ ]:
X_test_encoded = encoder.transform(Xts[['ESTU_PRGM_ACADEMICO']])

Xts['programa_encoded'] = X_test_encoded['ESTU_PRGM_ACADEMICO']

# Agregar la columna de encoding en el DataFrame original y eliminar la columna original
Xts = Xts.drop(columns=['ESTU_PRGM_ACADEMICO'])

In [ ]:
Xts = clean_dataset(Xts)

In [ ]:
preds = best_estimator.predict(pca.transform(Xts))
preds

array([1, 1, 4, ..., 3, 4, 4])

In [ ]:
preds = pd.Series(preds,name='RENDIMIENTO_GLOBAL')
preds = preds.map({1: "bajo", 2: "medio-bajo", 3: "medio-alto", 4: "alto"})
preds

,RENDIMIENTO_GLOBAL
0,bajo
1,bajo
2,alto
3,bajo
4,bajo
...,...
296781,bajo
296782,alto
296783,medio-alto
296784,alto


In [ ]:
#csv de prediccion
submission = pd.DataFrame([dts.ID,preds]).T
submission.head()

,ID,RENDIMIENTO_GLOBAL
0,550236,bajo
1,98545,bajo
2,499179,alto
3,782980,bajo
4,785185,bajo


In [ ]:
submission.to_csv('submission.csv',index=False)